# Kooperatif bilgilerinin cekilmesi
Turkiye'de kayitli kooperatiflerin bilgilerini cekmek icin bir Python 3 scriptini adim adim anlatiyorum. Veriler icin kaynagimiz Koopbis (https://koopbis.gtb.gov.tr/) sistemi.

Python'un kendi `csv` kutuphanesinin yanisira, HTTP request'lerini kolay yapabilmek icin requests kutuphanesini kullanacagiz. Eger ortaminizda kurulu degil ise kuralim.

In [ ]:
pip install requests

Kullanacagimiz kutuphaneleri import edelim.

In [ ]:
import csv
import requests

Verileri cekebilmek icin bir `session id`'sine ihtiyacimiz var. Bunu almak icin sayfaya bos bir `POST request` gonderiyoruz. 

In [ ]:
session = requests.Session()
response = session.post('https://koopbis.gtb.gov.tr/Portal/Home/KKSetFilterParameter')
session_id = session.cookies['ASP.NET_SessionId']

Aldigimiz session id'sini kullanarak, bir sonsuz dongu icerisinde sayfa sayfa sonuclari cekip ´csv´ dosyasina yaziyoruz. 

In [ ]:
with open('kooperatifler.csv', 'w') as f:
    writer = csv.writer(f)

    # kolon basliklarini yazdiralim
    writer.writerow([
        'Id',
        'Unvani',
        'Turu',
        'Kurulus',
        'Merkezi',
    ])

    p = 1
    while True:
        # sayfa numaralarini yazdiriralim ki bir hata olursa nerede kaldigimizi bilelim
        print(f'Sayfa: {p}')
        response = requests.post(
            'https://koopbis.gtb.gov.tr/Portal/Home/KoopKisaAraResult',
            data={
                'page': p,
                'pageSize': 1000,
            },
            cookies={
                'ASP.NET_SessionId': session_id
            }
        )
        for koop in response.json()['Data']['Items']:
            row = [
                koop['Id'],
                koop['Unvani'],
                koop['Turu'],
                koop['Kurulus'],
                koop['Merkezi'],
            ]
            writer.writerow(row)
        p += 1
        if p > response.json()['PageCount']:
            # tum kooperatifler cekildi
            break

Burada anlatilan islemler, kaynaktaki degisikliklere gore guncelligini kaybedebilir. Bir hata fark ederseniz lutfen bana (onur [at] genelizleyici.com) bildirmekten cekinmeyin. 